In [1]:
import numpy as np
from scipy.io import loadmat
import h5py
from EnsemblePursuit.EnsemblePursuit import EnsemblePursuit

In [2]:
#Data downloaded from https://janelia.figshare.com/articles/Whole-brain_light-sheet_imaging_data/7272617

In [3]:
beh_dat=loadmat('/media/maria/DATA1/Documents/ZebraFish/subject_1/data_full.mat')['data']['Behavior_full'][0][0]

In [4]:
stim_dat=loadmat('/media/maria/DATA1/Documents/ZebraFish/subject_1/data_full.mat')['data']['stim_full'][0][0]

In [5]:
hf = h5py.File('/media/maria/DATA1/Documents/ZebraFish/subject_1/TimeSeries.h5', 'r')
neural_dat=np.array(hf['CellResp'])

In [6]:
print(beh_dat.shape)
print(stim_dat.shape)
print(neural_dat.shape)

(5, 2880)
(1, 2880)
(2880, 83205)


In [7]:
def train_test_split(NT):
        nsegs = 20
        nt=NT
        nlen  = nt/nsegs
        ninds = np.linspace(0,nt-nlen,nsegs).astype(int)
        itest = (ninds[:,np.newaxis] + np.arange(0,nlen*0.25,1,int)).flatten()
        itrain = np.ones(nt, np.bool)
        itrain[itest] = 0
        return itrain, itest

In [8]:
itrain,itest=train_test_split(beh_dat.shape[1])
beh_train=beh_dat[:,itrain]
beh_test=beh_dat[:,itest]
stim_train=beh_dat[:,itrain]
stim_test=beh_dat[:,itest]
neural_train=neural_dat.T[:,itrain]
neural_test=neural_dat.T[:,itest]
print('Training set size: ',beh_train.shape[1])
print('Test set size: ',beh_test.shape[1])

Training set size:  2160
Test set size:  720


In [13]:
ep=EnsemblePursuit(n_components=100,lam=0.01,n_kmeans=100)
ep.fit(neural_train[:10000,:].T)

obtained 100 PCs in 1.6120 seconds
initialized 100 clusters with k-means in 3.4904 seconds
ensemble 0, time 5.51, nr neurons 3979, EV 0.0538
ensemble 25, time 39.21, nr neurons 851, EV 0.2080
ensemble 50, time 51.13, nr neurons 228, EV 0.2370
ensemble 75, time 58.97, nr neurons 41, EV 0.2530
ensemble 99, time 65.45, nr neurons 135, EV 0.2649
average sparsity is 0.0574


In [15]:
V=ep.components_
U=ep.weights

In [28]:
covM = np.matmul(beh_train, beh_train.T)
lam = 1e5 # regularizer
covM += lam*np.eye(beh_train.shape[0])
A = np.linalg.solve(covM, np.matmul(beh_train, V))
print(A.shape)
Vpred = np.matmul(A.T, beh_test)
beh_corr_lst=[]
print(V.shape)
print(Vpred.shape)
print(U.shape)
print(neural_test.shape)
V_g=U@neural_test[:10000,:]
print(V_g.shape)
for j in range(0,100):
    beh_corr_lst.append(np.corrcoef(Vpred[j,:],V.T[j,:])[0,1])

(5, 100)
(2160, 100)
(100, 720)
(10000, 100)
(83205, 720)


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 10000 is different from 100)